# Niejasności/głupoty/pytania:

- Jakie powinno być biznesowe kryterium sukcesu? +- godzina? Czy to ma sens tylko przy cichym założeniu że czas dostawy <24h? *Myślę, że zawsze ma czas, choć może raczej +/- 30 minut, skoro jednak operujemy w niewielkim oknie. Nie jestem pewna czy to nie jest jednak analityczne krterium sukcesu, a czy biznesowym nie powinno być coś w rodzaju "wzrostu zadowolenia klientów"*
- W jaki sposób ustalić na podstawie danych kryterium sukcesu? Zbudować już wstępny model żeby to zrobić? *To jedyny sposób, który znam - zasadniczo zbudowanie np. drzewa decyzyjnego, pomijając zupełnie brakujące dane, nie powinno zająć wiele czasu, a da niezłe pojęcie o tym, co da się uzyskać*
- Może głupie ale czy powinniśmy sprawdzić powiązanie wszystkich atrybutów ze zmienną celu i dopiero wtedy wybierać czy raczej od razu cisnąć na logikę to co się wydaje? *Zaczynałabym od policzenia powiązań/korelacji/informacji wzajemnej, a pote dopisywać do tego logikę*

# TODO:
- definicję problemu biznesowego,
- zdefiniowanie zadania/zadań modelowania i
- wszystkich założeń, zaproponowania kryteriów sukcesu),
- ii) analizę danych z perspektywy realizacji tych zadań (trzeba ocenić,
- czy dostarczone dane są wystarczające – może czegoś brakuje, może coś trzeba poprawić, domagać się
innych danych, …),
- wstępne dane wejściowe i zmienna celu
- wartości kryterium sukcesu na podstawie danych
-


- oszacowanie tego jaką dokładnie wartość powinna mieć miara, którą będziemy stosować na podstawie tego, co mamy
- no jakieś takie sensowne spięcie całości.
- komentarze odnośnie nowych danych (po połączeniu wszystkiego wciąż są brakujące wartości, dalej mamy dostawy <1h)
- macierz korelacji zmiennych zobrazowana na heatmapie (sns.heatmap)
- podzielić to co wiemy o danych na jakieś sensowne sekcje (nwm analiza starych danych, analiza nowych danych, sformułowanie problemów, etc)


- info o atrybutach czyli typ i wartosci
- sprawdzenie bledow i brakow
- czy dane sie wgl nadaja do modelowania
- informacja o zmiennej celu

## IUM - projekt
#### Autorzy:
Joanna Sokołowska, Rafał Uzarowicz
#### Zadanie:
3. “Wygląda na to, że nasze firmy kurierskie czasami nie radzą sobie z dostawami. Gdybyśmy
wiedzieli, ile taka dostawa dla danego zamówienia potrwa – moglibyśmy przekazywać tą
informację klientom."

In [ ]:
# Ustawienie środowiska

# Wstępne dane
data_v1_dir ="/data/data_v1"

deliveries_v1_path = data_v1_dir + "deliveries.jsonl"
sessions_v1_path = data_v1_dir + "sessions.jsonl"
users_v1_path = data_v1_dir + "users.jsonl"
products_v1_path = data_v1_dir + "products.jsonl"

# Pierwsza aktualizacja
data_v2_dir ="/data/data_v2"

deliveries_v2_path = data_v2_dir + "deliveries.jsonl"
sessions_v2_path = data_v2_dir + "sessions.jsonl"
users_v2_path = data_v2_dir + "users.jsonl"
products_v2_path = data_v2_dir + "products.jsonl"

# Tutaj może importy?

# Zadanie

### Tło biznesowe

#### Zadanie biznesowe
Przewidywanie przybliżonego czasu dostawy produktu.

#### Biznesowe kryterium sukcesu
Przewidywany przez system czas dostawy nie powinien zbytnio odstawać od rzeczywistego czasu dostawy.


### Model

#### Zadanie modelowania
Przewidywanie czasu dostawy przy użyciu regresji.

#### Analityczne kryterium sukcesu

NIE WIEM CO TUTEJ KURLA 

Kryteria oceny modelu mogą być takie jak zazwyczaj dla regresji np. MSE, lub absolute mean error. Dodatkowo można zamiast
jednego czasu dostawy przewidywać okno dostawy o założonej szerokości np. 2h i zastosować własną, niestandardową funkcję celu,
która traktuje wszystkie dostawy, które wpadły w okienko jako przewidziane poprawnie.

#### Zmienna celu
Przybliżona liczba godzin potrzebnych na dostawę produktu.


## Wstępna analiza zbioru danych

- krotki opis danych

- przeglad kolumn

- sprawdzenie powiazania wszystkiego ze zmienna celu?????????????????????????????????????????????????

- typ i wartosci wybranych atrybutow

- ustalenie wejscia do modelu ?

- sprawdzenie powiazania wejscia ze zmienna celu ?

# Analiza wstępnie wybranych danych

- sprawdzenie ilosci/jakości/reprezentatywności danych

- sprawdzenie brakow w danych

- sprawdzenie bledow w danych

- tutaj wnioski

- tutaj pytania ktore trzeba zadac do klienta


# Analiza pierwszej modyfikacji zbioru danych

- odpowiedzi od klienta/krotki opis aktualizacji danych

- sprawdzenie czy poprawiono poprzednie problemy

- sprawdzenie czy istnieja elementy odstajace

- sprawdzenie rozkladow?

- tutaj wnioski

- tutaj pytania ktore trzeba zadac do klienta